In [1]:
import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta

start = "2020-05-01"
duration = 91
today = datetime.today()

start_attr = start.split("-")
start_dt = datetime(int(start_attr[0]),int(start_attr[1]),int(start_attr[2]))

driver = webdriver.Chrome()
driver.implicitly_wait(20)
dataframe = pd.DataFrame()
for_day = start_dt

for i in range(1,duration):
    days = (for_day - today).days
    print("Extracting for Day {}".format(for_day))
    url = "https://www.google.com/flights?hl=en#flt=/m/06t2t.MAA.{};c:SGD;e:1;s:0;sd:1;t:f;tt:o".format(for_day.strftime("%Y-%m-%d"))
    print("Url {}".format(url))
    
    driver.get(url)
    time.sleep(5)
    s = BeautifulSoup(driver.page_source, features="html")

    datas = list()

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__carriers"})
    for div in divs:
        d = div.find("span", recursive=False).find("span", recursive=False)
        data.append(d.text.strip())
    #print(data)
    datas.append(data)

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__times"})
    for div in divs:
        d = div.find("span", recursive=False).find("span", recursive=False)
        data.append(d.text.strip())
    #print(data)
    datas.append(data)

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__stops"})
    for div in divs:
        d = div.find("div", recursive=False).find("span", recursive=False)
        data.append(d.text.strip())
    #print(data)
    datas.append(data)

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__airports"})
    for div in divs:
        d = div.find("span", recursive=False)
        data.append(d.text.strip())
    #print(data)
    datas.append(data)

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__airports"})
    for div in divs:
        d = div.findAll("span", recursive=False)
        data.append(d[1].text.strip())
    #print(data)
    datas.append(data)
    
    datas.append(['one-way' for x in range(0, len(datas[0]))])
    datas.append([for_day.strftime("%Y-%m-%d") for x in range(0, len(datas[0]))])
    datas.append([days for x in range(0, len(datas[0]))])

    data = list()
    divs = s.findAll("div", {"class": "gws-flights-results__collapsed-itinerary"})
    for div in divs:
        d = div.find("div", {"class": "gws-flights-results__itinerary-price"}, recursive=False)
        data.append((d.text.strip().replace("$", "")))
    #print(data)
    datas.append(data)
    
    df = pd.DataFrame(columns=['carrier','departure_time','stops','source','destination','itinerary_type','travel_date','days_before_travel','price'])
    df['carrier'] = datas[0]
    df['departure_time'] = datas[1]
    df['stops'] = datas[2]
    df['source'] = datas[3]
    df['destination'] = datas[4]
    df['itinerary_type'] = datas[5]
    df['travel_date'] = datas[6]
    df['days_before_travel'] = datas[7]
    df['price'] = datas[8]
    print(df.head())
    dataframe = pd.concat([dataframe, df],ignore_index=True, sort=False)
    for_day = for_day + timedelta(1)
    
driver.close()
dataframe.to_csv(r'data/SIN-MAA/airfare-SIN-MAA-{}.csv'.format(today.strftime("%Y-%m-%d")), index=False, header=True)

Extracting for Day 2020-05-01 00:00:00
Url https://www.google.com/flights?hl=en#flt=/m/06t2t.MAA.2020-05-01;c:SGD;e:1;s:0;sd:1;t:f;tt:o
              carrier departure_time     stops source destination  \
0             Silkair          07:40  Non-stop    SIN         MAA   
1  Singapore Airlines          20:25  Non-stop    SIN         MAA   
2              IndiGo          11:30  Non-stop    SIN         MAA   

  itinerary_type travel_date  days_before_travel              price  
0        one-way  2020-05-01                  12                972  
1        one-way  2020-05-01                  12                972  
2        one-way  2020-05-01                  12  Price unavailable  
Extracting for Day 2020-05-02 00:00:00
Url https://www.google.com/flights?hl=en#flt=/m/06t2t.MAA.2020-05-02;c:SGD;e:1;s:0;sd:1;t:f;tt:o
              carrier departure_time     stops source destination  \
0             Silkair          07:40  Non-stop    SIN         MAA   
1  Singapore Airlines          20